In [9]:
# part 1

import os, re, sys
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, operations = s.split("\n\n")


calculated = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}


operations = operations.split("\n")
dependencies = []
calculations = []
for operation in operations:
    args, outcome = operation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    calculations.append((args[1], outcome))



def calculate(deps, calcs):

    operation, res = calcs
    a1, a2 = deps
    if operation == 'OR':
        calculated[res] = calculated[a1] | calculated[a2]
    elif operation == 'XOR':
        calculated[res] = calculated[a1] ^ calculated[a2]
    elif operation == 'AND':
        calculated[res] = calculated[a1] & calculated[a2]


to_do = list(range(len(dependencies)))
while to_do:
    #print(to_do)
    for i in to_do:
        deps = dependencies[i]
        if deps[0] in calculated and deps[1] in calculated:
            calculate(deps, calculations[i])
            to_do.remove(i)
            break



count, total = 1, 0
for i in range(0, 100):
    val = 'z' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        break
    #result += str(int(calculated[val]))
    total += count * int(calculated[val])
    count *= 2

print(total)


59619940979346


In [10]:

# part 1

import os, re, sys
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, operations = s.split("\n\n")


calculated = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}


operations = operations.split("\n")
dependencies = []
calculations = []
for operation in operations:
    args, outcome = operation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    calculations.append((args[1], outcome))



def calculate(deps, calcs):

    operation, res = calcs
    a1, a2 = deps
    if operation == 'OR':
        calculated[res] = calculated[a1] | calculated[a2]
    elif operation == 'XOR':
        calculated[res] = calculated[a1] ^ calculated[a2]
    elif operation == 'AND':
        calculated[res] = calculated[a1] & calculated[a2]


to_do = list(range(len(dependencies)))
while to_do:
    #print(to_do)
    for i in to_do:
        deps = dependencies[i]
        if deps[0] in calculated and deps[1] in calculated:
            calculate(deps, calculations[i])
            to_do.remove(i)
            break



count, total = 1, 0
for i in range(0, 100):
    val = 'z' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        break
    #result += str(int(calculated[val]))
    total += count * int(calculated[val])
    count *= 2

print(total)




59619940979346


In [13]:




# generate a dependency tree:

affects = {}
comes_from = {}

for i in range(len(calculations)):
    a, b = dependencies[i]
    op, c = calculations[i]
    if affects.get(a):
        affects[a].append(c)
    else:
        affects[a] = [c] 

    if affects.get(b):
        affects[b].append(c)
    else:
        affects[b] = [c] 
    

    if comes_from.get(c):
        comes_from[c].append(b)
        comes_from[c].append(c)
    else:
        comes_from[c] = [a, b] 


results_from = {}
for i in range(len(calculations)):
    a, b = dependencies[i]
    op, c = calculations[i]
    results_from[c] = [a, b, op]



max_x, max_y, max_z = 0, 0, 0
for i in range(100):
    val = 'z' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_z = i
        break

for i in range(100):
    val = 'x' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_x = i
        break
for i in range(100):
    val = 'y' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_y = i
        break


print(max_x, max_y, max_z)


def calculate_var(name, max_i):
    total, count = 0, 1
    for i in range(max_i):
        val = name + ["0", ''][i >= 10] + str(i)
        total += int(calculated.get(val)) * count
        count *= 2
    return total

print(calculate_var('z', max_z))
print(calculate_var('y', max_y))
print(calculate_var('x', max_x))

z = calculate_var('z', max_z)
y = calculate_var('y', max_y)
x = calculate_var('x', max_x)


print(bin(z))
print(bin(x + y))
print(bin((x + y) ^ z))



false_neg = set()
false_pos = set()

for i in range(len(str(bin((x + y) ^ z)))-1, 1, -1):
    #print(i)
    print(str(bin((x + y) ^ z))[i], end='')

    if int(str(bin((x + y) ^ z))[i]):
        if int(str(bin(z))[i+4]):
            false_neg.add("z" + ["0", ''][i >= 10] + str(i))
        if int(str(bin(x + y))[i+4]):
            false_pos.add("z" + ["0", ''][i >= 10] + str(i))

print('')
print(false_neg)
print(false_pos)
wrong_bits = false_neg | false_pos
print(wrong_bits)


initial_wrong_bits = wrong_bits


45 45 46
59619940979346
28716883075641
30626032511001
0b1101100011100101011001011001110101001010010010
0b1101011111100011011001011001110100101001010010
0b111100000110000000000000000001100011000000
000000110001100000000000000000011000001111
{'z11', 'z02', 'z31', 'z36'}
{'z12', 'z04', 'z32', 'z05', 'z37', 'z03'}
{'z31', 'z12', 'z36', 'z04', 'z02', 'z32', 'z11', 'z05', 'z37', 'z03'}


In [12]:




def backtrack_wrongbits(false_neg, false_pos):
    if len(false_neg) == 4:
        if len(false_pos) == 4:
            print("we finished!")
            print(false_neg, false_pos)
            sys.exit()
    

    for false_n in false_neg:
        a, b, op = results_from[false_n]

        if a[0] in ['x', 'y']:
            if b[0] in ['x', 'y']:
                return
            else:

                if op == 'XOR':
                    backtrack_wrongbits()
                    
                continue
        
        
        elif b[0] in ['x', 'y']:  


            if op == "xor":
                backtrack_wrongbits
        
        n_false_pos = {}
        n_false_neg = {}

        if op == "OR":
            if calculated[a]:
                n_false_pos.add(a)
            if calculated[b]:
                n_false_pos.add(b)

            backtrack_wrongbits(false_neg | n_false_neg, false_pos | n_false_pos)

        elif op == "AND":
            if calculated[a]:
                backtrack_wrongbits(false_neg | {a}, false_pos)
            if calculated[b]:
                backtrack_wrongbits(false_neg | {b}, false_pos)

        elif op == "XOR":
            if calculated[a]:
                backtrack_wrongbits(false_neg, false_pos | {a})
                backtrack_wrongbits(false_neg | {b}, false_pos)
            if calculated[b]:
                backtrack_wrongbits(false_neg | {a}, false_pos)
                backtrack_wrongbits(false_neg, false_pos | {b})



    for false_p in false_pos:
        a, b, op = results_from[false_p]
        
        n_false_pos = {}
        n_false_neg = {}

        if op == "OR":
            if calculated[a]:
                n_false_pos.add(a)
            if calculated[b]:
                n_false_pos.add(b)

            backtrack_wrongbits(false_neg | n_false_neg, false_pos | n_false_pos)

        elif op == "AND":
            if calculated[a]:
                backtrack_wrongbits(false_neg | {a}, false_pos)
            if calculated[b]:
                backtrack_wrongbits(false_neg | {b}, false_pos)

        elif op == "XOR":
            if calculated[a]:
                backtrack_wrongbits(false_neg, false_pos | {a})
                backtrack_wrongbits(false_neg | {b}, false_pos)
            if calculated[b]:
                backtrack_wrongbits(false_neg | {a}, false_pos)
                backtrack_wrongbits(false_neg, false_pos | {b})





while not (len(false_neg) == 4 and len(false_pos) == 4):

    backtrack_wrongbits()


    










TypeError: backtrack_wrongbits() missing 2 required positional arguments: 'false_neg' and 'false_pos'

In [14]:
wrong_bits = initial_wrong_bits

wrong_bits = [(a, b) for a in false_neg for b in false_pos]


def comp(a, b, op):
    if op == 'XOR':
        return a ^ b
    elif op == 'OR':
        return a | b
    elif op == 'AND':
        return a & b
    


    
posss = [[0, 0], [0, 1], [1, 0], [1, 1]]

def backtrack_wrongbits(wrong_bits, depth = 0):
    print("\n" * 3, wrong_bits)
    total = 0
    for bit in wrong_bits:
        total += int(calculated[bit])
    if total == 0 and len(wrong_bits) <= 8:
        print(wrong_bits)
        sys.exit()


    count = 0
    for bit in wrong_bits:
        print(count)
        count += 1

        temp_wrong_bits = wrong_bits.copy()
        
        temp_wrong_bits.discard(bit)
        print("here", temp_wrong_bits)

        a, b, op = results_from[bit]
        res = calculated[bit]
        res_a, res_b = calculated[a], calculated[b]

        if a[0] in ['x', 'y']:
            if b[0] in ['x', 'y']:
                #print("in this weird case")
                backtrack_wrongbits(temp_wrong_bits, depth+1)
            else:
                if comp(res_a, not res_b, op) != res:
                    backtrack_wrongbits(temp_wrong_bits | {a}, depth+1)
        elif b[0] in ['x', 'y']:
            if comp(not res_a, res_b, op) != res:
                    backtrack_wrongbits(temp_wrong_bits | {b}, depth+1)

        else:
            print("no endpoints")
            if comp(not res_a, res_b, op) != res:
                print("a" ,end='')
                backtrack_wrongbits(temp_wrong_bits | {a}, depth+1)
            if comp(res_a, not res_b, op) != res:
                print("b" ,end='')
                backtrack_wrongbits(temp_wrong_bits | {b}, depth+1)
            if comp(not res_a, not res_b, op) != res:
                print("c" ,end='')
                backtrack_wrongbits(temp_wrong_bits | {a, b}, depth+1)

        

backtrack_wrongbits(wrong_bits)









 [('z11', 'z12'), ('z11', 'z04'), ('z11', 'z32'), ('z11', 'z05'), ('z11', 'z37'), ('z11', 'z03'), ('z02', 'z12'), ('z02', 'z04'), ('z02', 'z32'), ('z02', 'z05'), ('z02', 'z37'), ('z02', 'z03'), ('z31', 'z12'), ('z31', 'z04'), ('z31', 'z32'), ('z31', 'z05'), ('z31', 'z37'), ('z31', 'z03'), ('z36', 'z12'), ('z36', 'z04'), ('z36', 'z32'), ('z36', 'z05'), ('z36', 'z37'), ('z36', 'z03')]


KeyError: ('z11', 'z12')

In [ ]:
aas = [1, 2, 3, 4]
bees = [5, 6, 7, 8]

c = [{a, b} for a in aas for b in bees]
print(c)

[{1, 5}, {1, 6}, {1, 7}, {8, 1}, {2, 5}, {2, 6}, {2, 7}, {8, 2}, {3, 5}, {3, 6}, {3, 7}, {8, 3}, {4, 5}, {4, 6}, {4, 7}, {8, 4}]


In [46]:





# generate a dependency tree:

affects = {}
comes_from = {}

for i in range(len(calculations)):
    a, b = dependencies[i]
    op, c = calculations[i]
    if affects.get(a):
        affects[a].append(c)
    else:
        affects[a] = [c] 

    if affects.get(b):
        affects[b].append(c)
    else:
        affects[b] = [c] 
    

    if comes_from.get(c):
        comes_from[c].append(b)
        comes_from[c].append(c)
    else:
        comes_from[c] = [a, b] 


results_from = {}
for i in range(len(calculations)):
    a, b = dependencies[i]
    op, c = calculations[i]
    results_from[c] = [a, b, op]



max_x, max_y, max_z = 0, 0, 0
for i in range(100):
    val = 'z' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_z = i
        break

for i in range(100):
    val = 'x' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_x = i
        break
for i in range(100):
    val = 'y' + ["0", ''][i >= 10] + str(i)
    if calculated.get(val) is None:
        max_y = i
        break


#print(max_x, max_y, max_z)


def calculate_var(name, max_i):
    total, count = 0, 1
    for i in range(max_i):
        val = name + ["0", ''][i >= 10] + str(i)
        total += int(calculated.get(val)) * count
        count *= 2
    return total

#print(calculate_var('z', max_z))
#print(calculate_var('y', max_y))
#print(calculate_var('x', max_x))

z = calculate_var('z', max_z)
y = calculate_var('y', max_y)
x = calculate_var('x', max_x)


print(bin(z))
print(bin(x + y))
print(bin((x + y) ^ z))



false_neg = set()
false_pos = set()

for i in range(len(str(bin((x + y) ^ z)))-1, 1, -1):
    #print(i)
    print(str(bin((x + y) ^ z))[i], end='')

    if int(str(bin((x + y) ^ z))[i]):
        if int(str(bin(z))[i+4]):
            false_neg.add("z" + ["0", ''][i >= 10] + str(i + 4))
        if int(str(bin(x + y))[i+4]):
            false_pos.add("z" + ["0", ''][i >= 10] + str(i + 4))

print('')
print(false_neg)
print(false_pos)
wrong_bits = false_neg | false_pos
print(wrong_bits)



def comp(a, b, op):
    if op == 'XOR':
        return a ^ b
    elif op == 'OR':
        return a | b
    elif op == 'AND':
        return a & b
    

posss = [[0, 0], [0, 1], [1, 0], [1, 1]]

# all_wrong_bits = [(wrong_bits, {a, b}) for a in false_neg for b in false_pos]

initial_wrong_bits = wrong_bits

0b1101100011100101011001011001110101001010010010
0b1101011111100011011001011001110100101001010010
0b111100000110000000000000000001100011000000
000000110001100000000000000000011000001111
{'z06', 'z35', 'z15', 'z40'}
{'z16', 'z36', 'z09', 'z07', 'z08', 'z41'}
{'z16', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z08', 'z15', 'z41'}


In [42]:

all_done = []
all_endpoints = [initial_wrong_bits]

debug = False


i = 0
#while all_endpoints:

while all_endpoints:
    #print(len(all_endpoints))
    
    endpoint = all_endpoints.pop()

    #endpoint = all_endpoints[0]
    #del all_endpoints[0]
    #print(endpoint)
    all_done.append(endpoint)


    if len(endpoint) <= 8:
        print(endpoint)
        sys.exit()

    if debug: print("\n" * 5 + "new endpoint! " + str(i) + " ", len(endpoint))
    i += 1
    if i % 40 == 0: print('')
    print(endpoint)

    for bit in endpoint:

        print(len(all_endpoints), end=' ')
        

        #print("hi")
        
        new_endpoint = wrong_bits.copy()    
        new_endpoint.discard(bit)

            #print("here", temp_wrong_bits)

        a, b, op = results_from[bit]
        res = calculated[bit]
        res_a, res_b = calculated[a], calculated[b]

        if debug:print(a, b, op, res, res_a, res_b)

        assert(comp(res_a, res_b, op) == res)

        if a[0] in ['x', 'y'] or a in new_endpoint:
            if b[0] in ['x', 'y'] or b in new_endpoint:
                if debug: print('_', end='')
                newset = new_endpoint.copy()
                if newset not in all_done:
                    if debug: print('-succes')
                    all_endpoints.append(newset)
            else:
                if comp(res_a, not res_b, op) != res:
                    if debug: print('a1', end='')
                    newset = new_endpoint | {a}
                    if newset not in all_done: 
                        if debug: print('-succes')
                        all_endpoints.append(newset)
        elif b[0] in ['x', 'y'] or b in new_endpoint:
            
            if comp(not res_a, res_b, op) != res:
                if debug: print('b1', end='')
                #all_endpoints.append(new_endpoint | {b})
                newset = new_endpoint | {b}
                if newset not in all_done: 
                    if debug: print('-succes')
                    all_endpoints.append(newset)

        else:
            if comp(not res_a, res_b, op) != res:
                if debug: print('a2', end='')
                #all_endpoints.append(new_endpoint | {a})
                newset = new_endpoint | {a}
                if newset not in all_done: 
                    if debug:print('-succes')
                    all_endpoints.append(newset)
            if comp(res_a, not res_b, op) != res:
                if debug: print('b2', end='')
                #all_endpoints.append(new_endpoint | {b})
                newset = new_endpoint | {b}
                if newset not in all_done: 
                    if debug: print('-succes')
                    all_endpoints.append(newset)
            if comp(not res_a, not res_b, op) != res:  
                if debug: print('ab', end='')
                #all_endpoints.append(new_endpoint | {a, b})
                newset = new_endpoint | {a, b}
                if newset not in all_done: 
                    if debug:print('-succes')
                    all_endpoints.append(newset)





{'z31', 'z12', 'z36', 'z04', 'z02', 'z32', 'z11', 'z05', 'z37', 'z03'}
0 1 3 5 7 9 11 12 14 16 {'z16', 'wdm', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z08', 'z15', 'z41'}
17 19 22 24 27 29 31 33 35 37 39 {'z16', 'z36', 'z06', 'z09', 'z07', 'z40', 'cqq', 'z35', 'z08', 'z15'}
40 42 44 47 49 51 53 54 56 58 {'z16', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z08', 'dmh', 'z41'}
59 61 63 66 68 70 72 74 76 77 {'z16', 'nhs', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z08', 'z15'}
77 79 80 82 85 87 89 91 93 95 {'z16', 'vfj', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z08', 'z41'}
95 97 98 100 103 105 107 109 111 113 {'z16', 'z36', 'z06', 'z09', 'z07', 'z40', 'mpp', 'z35', 'z15', 'z41'}
112 114 116 119 121 123 125 128 130 130 {'mdg', 'z16', 'z36', 'z06', 'z09', 'z07', 'z40', 'z08', 'z15', 'z41'}
129 130 132 134 137 139 141 143 144 144 {'z16', 'hfd', 'z36', 'z06', 'z09', 'z07', 'z40', 'z35', 'z15', 'z41'}
143 145 146 148 151 153 155 157 158 158 {'z16', 'z36', 'z06', 'z09', 'z07', 'mhn', 'z40', 

In [45]:
b = [{1, 2, 3}, {4, 5}, {1}, {2}, {1, 2}]
c = [{1, 2, 3}, {4, 5}, {1}, {2}, {3}]
a = {1, 2}
print(a in b)
print(a in c)

True
False


In [ ]:

for i in range(len(all_done)):
    elem = all_done[i]
    print(len(elem), end=' ')
    if i % 40 == 0:
        print('')
    




In [ ]:
print(len())